In [1]:
import numpy as np
import pandas as pd

C:\Users\10322096\Miniconda3\envs\gen\lib\site-packages\numpy\__init__.py:138: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\10322096\Miniconda3\envs\gen\python.exe"
  * The NumPy version is: "1.19.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [78]:
csv_file = 'flowjo_stats_export.csv'

df = pd.read_csv(csv_file)
df.drop(axis=1, labels=['Unnamed: 7'], inplace=True)

In [17]:
df.describe()

,YG-582-A: %(+),YG-582-A: Median+,YG-582-A: Median-,YG-582-A: rSD-
count,58.000000,58.000000,58.000000,58.000000
mean,83.150000,7162.982759,126.266724,265.965517
std,24.251469,5022.595109,188.288316,127.027137
min,22.400000,913.000000,-778.000000,113.000000
25%,79.325000,2596.750000,44.975000,183.250000
50%,96.350000,6722.000000,154.000000,246.000000
75%,99.300000,10683.000000,233.500000,304.250000
max,99.800000,19302.000000,371.000000,920.000000


In [79]:
df.columns = ['filename', 'condition', 'concentration', 'pos_pct', 'median_pos', 'median_neg', 'rsd']

In [20]:
df.head()

,filename,condition,concentration,pos_pct,median_pos,median_neg,rsd
0,Specimen_002_A2_A02_003.fcs,4C,30,99.4,7652,350.0,151
1,Specimen_002_B2_B02_014.fcs,4C,60,98.5,3722,371.0,128
2,Specimen_002_C2_C02_025.fcs,4C,125,88.5,2232,264.0,184
3,Specimen_002_D2_D02_037.fcs,4C,250,94.3,2533,336.0,134
4,Specimen_002_E2_E02_047.fcs,4C,500,53.7,1436,102.0,242


In [80]:
# remove mean and SD rows
df = df.iloc[:len(df)-2, :]

In [28]:
def calc_stain_index(pos, neg, rsd):
    return (pos-neg)/(2*rsd)

def calc_SN(pos, neg):
    return pos/neg

In [81]:
df['stain_index'] = df.apply(lambda x: calc_stain_index(x['median_pos'], x['median_neg'], x['rsd']), axis = 1)

In [82]:
df['signal_noise'] = df.apply(lambda x: calc_SN(x['median_pos'], x['median_neg']), axis = 1)

In [66]:
df.head()

,filename,condition,concentration,pos_pct,median_pos,median_neg,rsd,stain_index,signal_noise
0,Specimen_002_A2_A02_003.fcs,4C,30,99.4,7652,350.0,151,24.178808,21.862857
1,Specimen_002_B2_B02_014.fcs,4C,60,98.5,3722,371.0,128,13.089844,10.032345
2,Specimen_002_C2_C02_025.fcs,4C,125,88.5,2232,264.0,184,5.347826,8.454545
3,Specimen_002_D2_D02_037.fcs,4C,250,94.3,2533,336.0,134,8.197761,7.538690
4,Specimen_002_E2_E02_047.fcs,4C,500,53.7,1436,102.0,242,2.756198,14.078431


In [83]:
df['concentration'] = df['concentration'].astype({'concentration': 'int32'})

In [67]:
# clip off 'y' to convert to numeric
df['condition'] = df['condition'].map(lambda x: x[:-1]).astype('float')

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 9 columns):
filename         56 non-null object
condition        56 non-null object
concentration    56 non-null int32
pos_pct          56 non-null float64
median_pos       56 non-null int64
median_neg       56 non-null float64
rsd              56 non-null int64
stain_index      56 non-null float64
signal_noise     56 non-null float64
dtypes: float64(4), int32(1), int64(2), object(2)
memory usage: 3.8+ KB


In [85]:
df = df.sort_values(['concentration'], ascending=[True])
df 

,filename,condition,concentration,pos_pct,median_pos,median_neg,rsd,stain_index,signal_noise
0,Specimen_002_A2_A02_003.fcs,4C,30,99.4,7652,350.00,151,24.178808,21.862857
28,Specimen_002_E5_E05_050.fcs,2.0y,30,98.5,12275,150.00,274,22.125912,81.833333
42,Specimen_002_C7_C07_030.fcs,4.0y,30,97.1,9203,109.00,401,11.339152,84.431193
49,Specimen_002_B8_B08_020.fcs,5.0y,30,99.8,15106,258.00,151,49.165563,58.550388
14,Specimen_002_G3_G03_071.fcs,1.0y,30,25.3,978,49.70,191,2.430105,19.678068
7,Specimen_002_H2_H02_081.fcs,0.5y,30,42.8,998,174.00,174,2.367816,5.735632
21,Specimen_002_F4_F04_060.fcs,1.5y,30,99.6,10826,333.00,139,37.744604,32.510511
35,Specimen_002_D6_D06_041.fcs,3.0y,30,99.6,10254,204.00,238,21.113445,50.264706
22,Specimen_002_G4_G04_072.fcs,1.5y,60,37.4,1246,-35.50,321,1.996106,-35.098592
15,Specimen_002_H3_H03_082.fcs,1.0y,60,49.5,1147,149.00,205,2.434146,7.697987


In [86]:
sorter = ['4C', '0.5y', '1.0y', '1.5y', '2.0y', '3.0y', '4.0y', '5.0y']

df.condition = df.condition.astype("category")
df.condition.cat.set_categories(sorter, inplace=True)

df

,filename,condition,concentration,pos_pct,median_pos,median_neg,rsd,stain_index,signal_noise
0,Specimen_002_A2_A02_003.fcs,4C,30,99.4,7652,350.00,151,24.178808,21.862857
28,Specimen_002_E5_E05_050.fcs,2.0y,30,98.5,12275,150.00,274,22.125912,81.833333
42,Specimen_002_C7_C07_030.fcs,4.0y,30,97.1,9203,109.00,401,11.339152,84.431193
49,Specimen_002_B8_B08_020.fcs,5.0y,30,99.8,15106,258.00,151,49.165563,58.550388
14,Specimen_002_G3_G03_071.fcs,1.0y,30,25.3,978,49.70,191,2.430105,19.678068
7,Specimen_002_H2_H02_081.fcs,0.5y,30,42.8,998,174.00,174,2.367816,5.735632
21,Specimen_002_F4_F04_060.fcs,1.5y,30,99.6,10826,333.00,139,37.744604,32.510511
35,Specimen_002_D6_D06_041.fcs,3.0y,30,99.6,10254,204.00,238,21.113445,50.264706
22,Specimen_002_G4_G04_072.fcs,1.5y,60,37.4,1246,-35.50,321,1.996106,-35.098592
15,Specimen_002_H3_H03_082.fcs,1.0y,60,49.5,1147,149.00,205,2.434146,7.697987


In [87]:
df.sort_values(["condition"])

,filename,condition,concentration,pos_pct,median_pos,median_neg,rsd,stain_index,signal_noise
0,Specimen_002_A2_A02_003.fcs,4C,30,99.4,7652,350.00,151,24.178808,21.862857
6,Specimen_002_G2_G02_070.fcs,4C,2000,22.4,913,43.40,183,2.375956,21.036866
5,Specimen_002_F2_F02_058.fcs,4C,1000,96.6,4251,338.00,134,14.600746,12.576923
4,Specimen_002_E2_E02_047.fcs,4C,500,53.7,1436,102.00,242,2.756198,14.078431
2,Specimen_002_C2_C02_025.fcs,4C,125,88.5,2232,264.00,184,5.347826,8.454545
1,Specimen_002_B2_B02_014.fcs,4C,60,98.5,3722,371.00,128,13.089844,10.032345
3,Specimen_002_D2_D02_037.fcs,4C,250,94.3,2533,336.00,134,8.197761,7.538690
10,Specimen_002_C3_C03_026.fcs,0.5y,250,96.0,3755,274.00,188,9.257979,13.704380
12,Specimen_002_E3_E03_048.fcs,0.5y,1000,63.1,1887,22.10,305,3.057213,85.384615
9,Specimen_002_B3_B03_015.fcs,0.5y,125,99.4,5925,307.00,113,24.858407,19.299674


In [32]:
import matplotlib.pyplot as plt

In [ ]:
# create a color palette
palette = plt.get_cmap('Set1')
 
# multiple line plot
num=0

for column in df.drop(['filename','condition', 'concentration'], axis=1):
    num+=1
    plt.plot(df['concentration'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

plt.legend(loc=2, ncol=2)
 
# Add titles
plt.title("A (bad) Spaghetti plot", loc='left', fontsize=12, fontweight=0, color='orange')
plt.xlabel("Time")
plt.ylabel("Score")
